# Exploring Hacker News post

## What is Hacker News?

Hacker news is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as 'posts') are voted and commented upon similar to reddit. Hacker news is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

## Dataset

The [dataset](https://www.kaggle.com/hacker-news/hacker-news-posts) has been reduced from almost 300,000 rows -> 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

Below are descriptions of the columns:

* `id` - The unique identifier from Hacker News for the post
* `title` - The title of the post
* `url` - The URL that the posts links to, if it the post has a URL
* `num_points` - The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* `num_comments` - The number of comments that were made on the post
* `author` - The username of the person who submitted the post
* `created_at` - The date and time at which the post was submitted

### Reading the dataset

Let us proceed reading the `hacker_news.csv` file in as a list of lists, and assign the result to a variable hn. Then we split the header with the contents, and display the first five rows of `hn`.


In [1]:
from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
headers = hn[0]
hn = hn[1:]

hn[:5]

[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

Let's confirm if we have split the header properly.

In [2]:
headers

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

Now we're ready to filter our data. Since we're only concerned with posts whose titles begin with `Ask HN` or `Show HN`, we'll create new lists of lists containing just the data for those titles.

In [3]:
ask_posts, show_posts, other_posts = [], [], []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


Let's check the contents of both `ask_posts` and `show_posts`

In [4]:
ask_posts[0:4]

[['12296411',
  'Ask HN: How to improve my personal website?',
  '',
  '2',
  '6',
  'ahmedbaracat',
  '8/16/2016 9:55'],
 ['10610020',
  'Ask HN: Am I the only one outraged by Twitter shutting down share counts?',
  '',
  '28',
  '29',
  'tkfx',
  '11/22/2015 13:43'],
 ['11610310',
  'Ask HN: Aby recent changes to CSS that broke mobile?',
  '',
  '1',
  '1',
  'polskibus',
  '5/2/2016 10:14'],
 ['12210105',
  'Ask HN: Looking for Employee #3 How do I do it?',
  '',
  '1',
  '3',
  'sph130',
  '8/2/2016 14:20']]

In [5]:
show_posts[0:4]

[['10627194',
  'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform',
  'https://iot.seeed.cc',
  '26',
  '22',
  'kfihihc',
  '11/25/2015 14:03'],
 ['10646440',
  'Show HN: Something pointless I made',
  'http://dn.ht/picklecat/',
  '747',
  '102',
  'dhotson',
  '11/29/2015 22:46'],
 ['11590768',
  'Show HN: Shanhu.io, a programming playground powered by e8vm',
  'https://shanhu.io',
  '1',
  '1',
  'h8liu',
  '4/28/2016 18:05'],
 ['12178806',
  'Show HN: Webscope  Easy way for web developers to communicate with Clients',
  'http://webscopeapp.com',
  '3',
  '3',
  'fastbrick',
  '7/28/2016 7:11']]

Next, let's determine if ask posts or show posts receive more comments on average.

In [6]:
# First, we define functions to get the total & average comments
def get_total_comments(posts: [list], idx: int) -> int:
    return sum(int(post[idx]) for post in posts)

def get_average_comment(posts: [list], idx: int) -> float:
    return get_total_comments(posts, idx) / len(posts)

idx_comments = 4

# Now lets find the total number of comments in ask posts & show posts
print('Average comments on Ask post: {:.2f}'.format(get_average_comment(ask_posts, idx_comments)))
print('Average comments on Show post: {:.2f}'.format(get_average_comment(show_posts, idx_comments)))

Average comments on Ask post: 14.04
Average comments on Show post: 10.32


We can see that on average, Ask post receive more comments than Show post. This is probably because that Ask posts are more likely to attract comments.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

In [7]:
import datetime as dt

result_list = [] # list of lists
# total_comments = get_total_comments(ask_posts, idx_comments)

idx_created_at = 6
idx_date = 0
for post in ask_posts:
    result_list.append(
        [post[idx_created_at], int(post[idx_comments])]
    )

counts_by_hour, comments_by_hour = {}, {}

date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    date = row[0]
    comment = row[1]
    time = dt.datetime.strptime(date, date_format).strftime('%H')   
    if time in counts_by_hour:
        counts_by_hour[time] += 1
        comments_by_hour[time] += comment
    else:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comment

In [8]:
counts_by_hour

{'00': 55,
 '01': 60,
 '02': 58,
 '03': 54,
 '04': 47,
 '05': 46,
 '06': 44,
 '07': 34,
 '08': 48,
 '09': 45,
 '10': 59,
 '11': 58,
 '12': 73,
 '13': 85,
 '14': 107,
 '15': 116,
 '16': 108,
 '17': 100,
 '18': 109,
 '19': 110,
 '20': 80,
 '21': 109,
 '22': 71,
 '23': 68}

In [9]:
comments_by_hour

{'00': 447,
 '01': 683,
 '02': 1381,
 '03': 421,
 '04': 337,
 '05': 464,
 '06': 397,
 '07': 267,
 '08': 492,
 '09': 251,
 '10': 793,
 '11': 641,
 '12': 687,
 '13': 1253,
 '14': 1416,
 '15': 4477,
 '16': 1814,
 '17': 1146,
 '18': 1439,
 '19': 1188,
 '20': 1722,
 '21': 1745,
 '22': 479,
 '23': 543}

Now let's calculate the average number of comments per post for post created during each hour of the day

In [10]:
# avg_by_hour = [[hour, avg_comment_perpost]]
avg_by_hour = []

for hour in comments_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour] / counts_by_hour[hour]])

avg_by_hour

[['14', 13.233644859813085],
 ['00', 8.127272727272727],
 ['11', 11.051724137931034],
 ['10', 13.440677966101696],
 ['06', 9.022727272727273],
 ['04', 7.170212765957447],
 ['23', 7.985294117647059],
 ['13', 14.741176470588234],
 ['21', 16.009174311926607],
 ['15', 38.5948275862069],
 ['05', 10.08695652173913],
 ['03', 7.796296296296297],
 ['18', 13.20183486238532],
 ['22', 6.746478873239437],
 ['07', 7.852941176470588],
 ['09', 5.5777777777777775],
 ['16', 16.796296296296298],
 ['12', 9.41095890410959],
 ['08', 10.25],
 ['17', 11.46],
 ['02', 23.810344827586206],
 ['19', 10.8],
 ['20', 21.525],
 ['01', 11.383333333333333]]

In [11]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)
# sorted_swap

print('Top 5 hours for Ask Posts Comments')
for avg, hr in sorted_swap[:5]:
    print('{} -> {:.2f} average comments per post'.format(
        dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg))

Top 5 hours for Ask Posts Comments
15:00 -> 38.59 average comments per post
02:00 -> 23.81 average comments per post
20:00 -> 21.52 average comments per post
16:00 -> 16.80 average comments per post
21:00 -> 16.01 average comments per post


According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

Therefore, you should post between 3PM - 4PM EST to get higher chance of receiving comments.

# Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.